In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
#from langchain_community.tools import DuckDuckGoSearchRun
from textwrap import dedent
from langchain.agents import tool
from dotenv import load_dotenv
from dotenv import load_dotenv
import os
from crewai_tools import SerperDevTool
load_dotenv()
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
# 1. Định nghĩa công cụ (ở đây dùng DuckDuckGo Search của LangChain)
search_tool = SerperDevTool(n_results=5, search_type= "search", hl='vi', gl='vn', page="1")
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# create a senior researcher agent with memory and verbose mode
llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.7,
    api_key=GEMINI_API_KEY)

# 2. Định nghĩa các agentkk
researcher = Agent(
    role='Nhà nghiên cứu', # Vai trò của agent
    goal='Tìm kiếm thông tin về chủ đề được giao', # Mục tiêu của agent
    backstory=dedent("""Bạn là một nhà nghiên cứu chuyên nghiệp, 
    có khả năng tìm kiếm và tổng hợp thông tin từ nhiều nguồn khác nhau."""), # Lý do agent tham gia vào công việc
    verbose=True,  # In ra các thông tin chi tiết trong quá trình chạy
    allow_delegation=False, # Không cho phép ủy thác công việc
    tools=[search_tool] ,
    llm= llm
)

writer = Agent(
    role='Người viết',
    goal='Viết một đoạn tóm tắt ngắn gọn và súc tích về thông tin đã tìm được',
    backstory=dedent("""Bạn là một người viết chuyên nghiệp,
    có khả năng tóm tắt thông tin một cách rõ ràng và dễ hiểu."""),
    verbose=True,
    allow_delegation=False,
    tools=[search_tool] ,
    llm= llm
)

# 3. Định nghĩa các task
task1 = Task( 
    description='Tìm kiếm thông tin về {topic}',
    agent=researcher, # Agent thực hiện task này,
    expected_output="Một danh sách các thông tin quan trọng về {topic}, bao gồm định nghĩa, tính năng, cách sử dụng, và các ví dụ.",
    tools=[search_tool]
)

task2 = Task(
    description='Viết một đoạn tóm tắt ngắn gọn về {topic}',
    agent=writer,
    expected_output="Một đoạn văn bản tóm tắt ngắn gọn giới thiệu về {topic}, nêu bật các điểm chính và lợi ích của nó." ,
    async_execution=False,
    tools=[search_tool],
    output_file='output.md'
)

# 4. Tạo crew và chọn quy trình làm việc (sequential)
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=True,  # In ra thông tin chi tiết ở cấp độ crew
    process=Process.sequential,  # Quy trình tuần tự 
    #process = Process.hierarchical # Quy trình phân cấp
)

# 5. Thực thi
result = crew.kickoff(inputs={'topic': 'Trí tuệ nhân tạo trong giáo dục'})
print("Kết quả cuối cùng:")
print(result)

2025-02-10 15:50:31,239 - 140116407480704 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Nhà nghiên cứu
## Task: Tìm kiếm thông tin về Trí tuệ nhân tạo trong giáo dục


# Agent: Nhà nghiên cứu
## Thought: Okay, I need to find comprehensive information about Artificial Intelligence in education, including its definition, features, uses, and examples.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Artificial Intelligence in education: definition, features, uses, examples\"}"
## Tool Output: 
{'searchParameters': {'q': 'Artificial Intelligence in education: definition, features, uses, examples', 'type': 'search', 'num': 5, 'engine': 'google'}, 'organic': [{'title': '39 Examples of Artificial Intelligence in Education', 'link': 'https://onlinedegrees.sandiego.edu/artificial-intelligence-education/', 'snippet': 'AI tools help teachers manage classroom behavior and engagement. For example, Classcraft uses AI to gamify classroom management, tracking student behavior and ...', 'position': 1, 'sitelinks': [{'title': 'Benefits of AI in 